In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging
from sqlalchemy import modifier

load_dotenv()
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


프롬프트 템플릿에 {num}이라는 입력 변수

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template("{num}의 10배는?")
llm = ChatOpenAI(temperature=0)
chain = prompt | llm

chain.invoke() 메소드로 실행
입력 데이터의 타입은 키와 값의 쌍으로 구성된 딕셔너리여야 한다!

In [3]:
chain.invoke({"num": 5})

AIMessage(content='50', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D4t0ljw2UUFWHghOkR9UUD426A3xc', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c1fa0-a295-7f11-92ab-15e070314957-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

RunnablePassThrough을 사용하면 입력 받은 값 그대로 출력

In [4]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke({"num": 10})

{'num': 10}

RunnablePassThrough를 사용하면 사용자는 키 값을 따로 기억할 필요 없이 단순히 값만 전달해도 자동으로 딕셔너리 형태로 변환되어 결과를 얻을 수 있다

In [5]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()
# 10이라는 값만 입력해도 RunnablePassthrough를 통해 {"num": 10}으로 변환되어 프롬프트로 전달
runnable_chain.invoke({"num": 10})

AIMessage(content='100입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D4t0mnyy4iBW7ItMebexAv8hKb9yd', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c1fa0-a708-7cc3-a733-81ea49030339-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

{"num": 1}이라는 딕셔너리가 입력되고, 입력된 값이 그대로 출력

In [6]:
RunnablePassthrough().invoke({"num": 1})

{'num': 1}

`RunnablePassthrough.assign()`은 입력된 값의 키-값 쌍과 새롭게 할당된 키-값 쌍을 결합하는 기능

In [7]:
# 입력 키: num, 할당(assign) 키: new_num
(RunnablePassthrough().assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

RunnableParallel 인스턴스 생성하고, passed, extra, modified 키에 따라 병렬 처리

In [8]:
from langchain_core.runnables import RunnableParallel

runnable = RunnableParallel(
    passed = RunnablePassthrough(),
    extra = RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified = lambda x: x["num"] + 1
)

In [9]:
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

체인에도 RunnableParallel 적용 가능

In [10]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)

In [11]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='서울특별시입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 19, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D4t0nUnmZzSO7b3L3BhhuB97bFFug', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c1fa0-a926-7522-904e-fa3b24542825-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 19, 'output_tokens': 10, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'area': AIMessage(content='대한민국의 총 면적은 약 100,363.4 km² 입니다.', additional_kwargs={'refusal': None}, response_metadata={'toke

RunnableLambda를 사용하여 사용자 정의 함수 매핑

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from datetime import datetime

def get_today(a):
    return datetime.today().strftime("%b-%d")

get_today(None)

'Feb-03'

In [13]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

prompt = PromptTemplate.from_template("{today}가 생일인 유명인 {n}명을 나열하세요. 생년월일을 표기해 주세요")
llm = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
print(chain.invoke(3))

다음은 2월 3일이 생일인 유명인 3명입니다:

1. 게리 올드먼 (Gary Oldman) - 1958년 2월 3일  
2. 모니카 루인스키 (Monica Lewinsky) - 1973년 2월 3일  
3. 토마스 페인 (Thomas Paine) - 1737년 2월 3일


- **itemgetter** 를 사용하여 딕셔너리에서 특정 키의 값을 추출하고, 이를 사용자 정의 함수에 매핑
- itemgetter와 RunnableLambda를 사용해 두 개의 문장의 길이를 계산하고 그 값을 OpenAI 모델에 전달하는 체인

In [15]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) + len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

In [17]:
prompt = ChatPromptTemplate.from_template("{a} + {b}는 무엇인가요?")
model = ChatOpenAI()

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

chain.invoke({"word1": "hello", "word2": "world"})

AIMessage(content='5 + 10은 15입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 20, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D4t7PEvgUScETnPYkFB3LtoAPwnYH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c1fa6-ec04-7412-9725-c06ef29ea6b0-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 20, 'output_tokens': 9, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

입력값을 처리하고 프롬프트를 모델에 전달하는 체인 작성